In [1]:
!git clone https://github.com/ultralytics/yolov5.git

# /content/drive/MyDrive/wheat/pesos

Cloning into 'yolov5'...
remote: Enumerating objects: 8700, done.
remote: Counting objects: 100% (414/414), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 8700 (delta 256), reused 260 (delta 142), pack-reused 8286
Receiving objects: 100% (8700/8700), 9.69 MiB | 24.02 MiB/s, done.
Resolving deltas: 100% (5996/5996), done.


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Aug  2 18:53:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -r yolov5/requirements.txt

     |████████████████████████████████| 636 kB 8.7 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!unzip /content/drive/MyDrive/Bosh_AI_test/dataset_gt_split_bosh.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset_gt/train/aclass_train_imageID_26_GT_5_0_0.txt  
  inflating: dataset_gt/train/aclass_train_imageID_39_GT_1_0_3.txt  
  inflating: dataset_gt/train/aclass_train_imageID_46_GT_3_0_2.txt  
  inflating: dataset_gt/train/aclass_train_imageID_62_GT_2_4_3.txt  
  inflating: dataset_gt/train/aclass_train_imageID_65_GT_0_3_0.txt  
  inflating: dataset_gt/train/aclass_train_imageID_68_GT_0_3_0.txt  
  inflating: dataset_gt/train/aclass_train_imageID_79_GT_1_4_0.txt  
  inflating: dataset_gt/train/aclass_train_imageID_93_GT_1_3_0.txt  
  inflating: dataset_gt/train/aclass_train_imageID_94_GT_2_4_1.txt  
  inflating: dataset_gt/train/aclass_train_imageID_97_GT_3_0_1.txt  
  inflating: dataset_gt/train/aclass_train_imageID_110_GT_5_4_0.txt  
  inflating: dataset_gt/train/aclass_train_imageID_121_GT_0_0_4.txt  
  inflating: dataset_gt/train/aclass_train_imageID_139_GT_4_5_0.txt  
  inflating: dataset_gt/train/aclass_train_imageI

In [5]:
!unzip /content/drive/MyDrive/Bosh_AI_test/dataset_split_bosh.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/aclass_train_imageID_7_GT_5_0_0.png  
  inflating: dataset/train/aclass_train_imageID_32_GT_6_5_0.png  
  inflating: dataset/train/aclass_train_imageID_37_GT_3_0_3.png  
  inflating: dataset/train/aclass_train_imageID_40_GT_1_0_0.png  
  inflating: dataset/train/aclass_train_imageID_50_GT_3_0_5.png  
  inflating: dataset/train/aclass_train_imageID_54_GT_3_0_1.png  
  inflating: dataset/train/aclass_train_imageID_76_GT_0_4_3.png  
  inflating: dataset/train/aclass_train_imageID_110_GT_5_4_0.png  
  inflating: dataset/train/aclass_train_imageID_114_GT_5_6_0.png  
  inflating: dataset/train/aclass_train_imageID_125_GT_4_4_3.png  
  inflating: dataset/train/aclass_train_imageID_146_GT_6_2_0.png  
  inflating: dataset/train/aclass_train_imageID_147_GT_2_2_4.png  
  inflating: dataset/train/aclass_train_imageID_154_GT_4_3_0.png  
  inflating: dataset/train/aclass_train_imageID_155_GT_0_5_0.png  
  inflating: datase

In [6]:
import torch
#from IPython.display import Image as img_disp  # for displaying images
import os 
import random
import shutil
import pandas as pd
from glob import glob
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

# os.mkdir('./images')

In [7]:
def extract_from_txt(txt, task='train/'):
    info_dict = {}
    info_dict['bboxes'] = []
    img_name = txt.split('/')[-1].split('.')[0]
    img = cv2.imread('./images/' + task + img_name + '.png')
    image_size = img.shape
    label_true = pd.read_table(txt, names=['cls'])
    box_coords_true = list()
    for box in label_true['cls']:
        label, x1, y1, x2, y2 = list(map(int, box.split(',')))
        box_coords_true.append([label, x1, y1, x2, y2])
    info_dict['filename'] = img_name + '.png'
    info_dict['image_size'] = image_size
    
    if box_coords_true:
        
        for box in box_coords_true:
            b = {}
            b['class'] = box[0] - 1
            xmin, ymin, xmax, ymax = box[1:]
            b['xmin'] = xmin
            b['ymin'] = ymin
            b['xmax'] = xmax
            b['ymax'] = ymax

            info_dict['bboxes'].append(b)
    
    return info_dict

In [8]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"head": 0}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict, task):
    print_buffer = []
    
    if info_dict["bboxes"]:
    
        # For each bounding box
        for b in info_dict["bboxes"]:
            try:
                class_id = b["class"]
            except KeyError:
                print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())

            # Transform the bbox co-ordinates as per the format required by YOLO v5
            b_center_x = (b["xmin"] + b["xmax"]) / 2 
            b_center_y = (b["ymin"] + b["ymax"]) / 2
            b_width    = (b["xmax"] - b["xmin"])
            b_height   = (b["ymax"] - b["ymin"])

            # Normalise the co-ordinates by the dimensions of the image
            image_h, image_w, image_c = info_dict["image_size"]  
            b_center_x /= image_w 
            b_center_y /= image_h 
            b_width    /= image_w 
            b_height   /= image_h 

            #Write the bbox details to the file 
            print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, min(b_center_x, 1.), min(b_center_y, 1.), b_width, b_height))
    
    if not os.path.isdir('./' + task):         
        os.mkdir('./' + task)
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("./", task, info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [10]:
# Convert and save the annotations
task = 'train/'
all_txt = glob('./dataset_gt/' + task + '*.txt')
for txt in tqdm(all_txt):
    info_dict = extract_from_txt(txt, task=task)
    convert_to_yolov5(info_dict, task)

# Convert and save the annotations
task = 'val/'
all_txt = glob('./dataset_gt/' + task + '*.txt')
for txt in tqdm(all_txt):
    info_dict = extract_from_txt(txt, task=task)
    convert_to_yolov5(info_dict, task)

100%|██████████| 1200/1200 [00:36<00:00, 32.98it/s]


In [11]:
# Read images and annotations
annotations_train = [os.path.join('./train', x) for x in os.listdir('./train') if x[-3:] == "txt"]
annotations_val = [os.path.join('./val', x) for x in os.listdir('./val') if x[-3:] == "txt"]

In [12]:
print(len(annotations_train))
print(len(annotations_val))

10800
1200


In [13]:
os.mkdir('./labels')

#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    if not os.path.isdir(destination_folder):
         os.mkdir(destination_folder)
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(annotations_train, 'labels/train/')
move_files_to_folder(annotations_val, 'labels/val/')

In [14]:
%cd yolov5/

/content/yolov5


In [15]:
!ls

CONTRIBUTING.md  Dockerfile  LICENSE	requirements.txt  utils
data		 export.py   models	train.py	  val.py
detect.py	 hubconf.py  README.md	tutorial.ipynb


In [17]:
!bash data/scripts/download_weights.sh

100% 14.1M/14.1M [00:00<00:00, 23.3MB/s]

100% 41.1M/41.1M [00:00<00:00, 50.2MB/s]

100% 90.2M/90.2M [00:01<00:00, 68.7MB/s]

100% 168M/168M [00:01<00:00, 90.2MB/s]



In [18]:
!python train.py --img 640 --cfg models/yolov5s.yaml --hyp data/hyps/hyp.scratch.yaml --batch 32 --epochs 5 --data /content/drive/MyDrive/Bosh_AI_test/cust_bosh.yaml --weights yolov5s.pt --workers 4 --name cust_bosh --device 0

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=/content/drive/MyDrive/Bosh_AI_test/cust_bosh.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=5, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=4, project=runs/train, entity=None, name=cust_bosh, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-345-g2d99063 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=2.0, hsv_h=0.015, hsv_s=0.7, hsv_